Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple

Function to calculate IoU

In [3]:
@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

Enter the bounding box for the ground truth (box 1) followed by the bounding box of the predicted model(box 2)

### Image - lucas-hoang-rquNBlHgSWg-unsplash_jpg.rf.44d9d1748a02fdd36348b92739b7d130.jpg

In [4]:
box1 = np.array([0,0,401.39,337.78])
box2 = np.array([0,0,403.78,335.1])
calculate_iou(box1, box2, form='coco')

0.9862775955880806

Calculate the IoU by filling the function with the bounding boxes

In [5]:
box1 = np.array([0,41,216.67,299.7])
box2 = np.array([0,103,130.04,237.3])
calculate_iou(box1, box2, form='coco')

0.47708517494340136

In [6]:
box1 = np.array([1,414,214.3,359.64])
box2 = np.array([0,410,211.79,362.45])
calculate_iou(box1, box2, form='coco')

0.965399302646347

In [7]:
box1 = np.array([170,293,242.46,289.13])
box2 = np.array([176,307,235.75,271.6])
calculate_iou(box1, box2, form='coco')

0.9136830820214129

In [8]:
box1 = np.array([366,327,191.92,260.21])
box2 = np.array([372,314,184.01,271.4])
calculate_iou(box1, box2, form='coco')

0.9089702118289735

### Image - vladimir-fedotov-0f6d8tYEljU-unsplash_jpg.rf.778319bde7a7969eb32e3946c774235c.jpg

In [12]:
box1 = np.array([0,0,332.51,148])
box2 = np.array([1,1,331.84,176.89])
calculate_iou(box1, box2, form='coco')

0.8244452288770718

In [13]:
box1 = np.array([212,0,353.03,350.12])
box2 = np.array([214,0,348.36,345.48])
calculate_iou(box1, box2, form='coco')

0.9737684889567648

In [14]:
box1 = np.array([519,0,429.35,227.94])
box2 = np.array([554,0,379.92,258.62])
calculate_iou(box1, box2, form='coco')

0.7912809131951756

In [15]:
box1 = np.array([0,61,284.67,178.53])
box2 = np.array([1,57,340.13,181.15])
calculate_iou(box1, box2, form='coco')

0.8087163794828006

### Image - henley-design-studio-2ywzLqu-cuk-unsplash_jpg.rf.3a8b65abbadba355ad8e51c7b95790e9.jpg

In [16]:
box1 = np.array([509,122,455.88,488.49])
box2 = np.array([516,128,436,481.87])
calculate_iou(box1, box2, form='coco')

0.9435516754438854

### Image - audrey-martin-FJpHcqMud_Y-unsplash_jpg.rf.bba62c8515242f48efd6547a835ab5e7.jpg

In [17]:
box1 = np.array([219,29,211.79,229])
box2 = np.array([219,30,213.67,223.03])
calculate_iou(box1, box2, form='coco')

0.965732695836177

In [18]:
box1 = np.array([109,157,174.83,167])
box2 = np.array([109,158,176.01,164.85])
calculate_iou(box1, box2, form='coco')

0.9807050700916426

In [19]:
box1 = np.array([287,213,156.17,160])
box2 = np.array([288,206,153.69,164.8])
calculate_iou(box1, box2, form='coco')

0.9309351503759401

In [20]:
box1 = np.array([178,251,186.81,189])
box2 = np.array([174,254,192.43,186.05])
calculate_iou(box1, box2, form='coco')

0.9558017936910117

In [21]:
box1 = np.array([24,307,192.84,190])
box2 = np.array([25,311,188.79,181.92])
calculate_iou(box1, box2, form='coco')

0.9376866871365813

In [22]:
box1 = np.array([205,373,157.15,165])
box2 = np.array([204,378,151.68,156.87])
calculate_iou(box1, box2, form='coco')

0.90666503097032

In [23]:
box1 = np.array([424,217,194.75,185])
box2 = np.array([426,218,191.87,183.13])
calculate_iou(box1, box2, form='coco')

0.975381510320109

In [24]:
box1 = np.array([325,367,207.18,216])
box2 = np.array([324,368,206.62,210.86])
calculate_iou(box1, box2, form='coco')

0.9644741965306302

### Image - ayla-verschueren-UjOi6URY-C8-unsplash_jpg.rf.ec1015a78a0be01d13f34430c3caaa93.jpg

In [25]:
box1 = np.array([1,490,252.39,322.46])
box2 = np.array([0,494,254.87,314.37])
calculate_iou(box1, box2, form='coco')

0.9657732913319614

In [26]:
box1 = np.array([218,121,287.68,296.99])
box2 = np.array([219,118,285.56,299.94])
calculate_iou(box1, box2, form='coco')

0.9826693450691127

In [27]:
box1 = np.array([1199,411,300.08,307.77])
box2 = np.array([1213,407,284.34,296.98])
calculate_iou(box1, box2, form='coco')

0.8913820936895052

In [28]:
box1 = np.array([1262,885,289.32,314.81])
box2 = np.array([1249,887,304.07,293.37])
calculate_iou(box1, box2, form='coco')

0.8899651407307889

### Image - siora-photography-1VDWmab-Glk-unsplash_jpg.rf.38c8ae8104a036377c0b74344aa14731.jpg

In [29]:
box1 = np.array([2272,0,1184.36,1278.52])
box2 = np.array([2269,0,1187.1,1578.07])
calculate_iou(box1, box2, form='coco')

0.8081103972823126

In [30]:
box1 = np.array([871,0,1404.28,1244.77])
box2 = np.array([877,0,1391.87,1218.2])
calculate_iou(box1, box2, form='coco')

0.9700291938437722

In [31]:
box1 = np.array([1559,570,1509.58,1928.34])
box2 = np.array([1555,570,1511.98,1888.75])
calculate_iou(box1, box2, form='coco')

0.9759113929384781

In [32]:
box1 = np.array([1196,1215,1155.31,1544.36])
box2 = np.array([1197,960,1191.84,1793.89])
calculate_iou(box1, box2, form='coco')

0.8278959642245994

### Image - anne-nygard-1hxeCZ-S65M-unsplash_jpg.rf.fb6926255b5e4e205c0d140f8707364e.jpg

In [22]:
box1 = np.array([412,343,56.24,65.18])
box2 = np.array([411,345,57.89,73.94])
calculate_iou(box1, box2, form='coco')

0.8113757445673373

In [23]:
box1 = np.array([358,357,81.43,57.03])
box2 = np.array([358,357,62.11,65.94])
calculate_iou(box1, box2, form='coco')

0.6850847490601363

In [24]:
box1 = np.array([436,379,67.81,58.51])
box2 = np.array([442,379,58.75,57.65])
calculate_iou(box1, box2, form='coco')

0.8557845033476044

In [25]:
box1 = np.array([381,386,56.62,52.59])
box2 = np.array([382,390,54.96,55.36])
calculate_iou(box1, box2, form='coco')

0.8004881869094719

In [26]:
box1 = np.array([419,423,43.74,50.74])
box2 = np.array([416,425,47.03,48.06])
calculate_iou(box1, box2, form='coco')

0.886396756392214

In [27]:
box1 = np.array([368,435,49.59,43.7])
box2 = np.array([366,426,50.11,52.06])
calculate_iou(box1, box2, form='coco')

0.7701725192855768

### Image - siamak-cEVBsYy3gxU-unsplash_jpg.rf.e07e16caf07207f22110b42ecf2d4320.jpg

In [28]:
box1 = np.array([0,85,79.28,212.69])
box2 = np.array([0,87,81.28,209.58])
calculate_iou(box1, box2, form='coco')

0.9618329866788217

In [29]:
box1 = np.array([60,9,171.75,200.47])
box2 = np.array([58,0,174.13,206.2])
calculate_iou(box1, box2, form='coco')

0.929100437987765

In [31]:
box1 = np.array([72,125,150.46,194.3])
box2 = np.array([67,124,154.49,191.81])
calculate_iou(box1, box2, form='coco')

0.9404068972477901

### Image - gift-habeshaw-kBhmgL0ngHw-unsplash_jpg.rf.4bcc3183f16d6a93d61822377b341c06.jpg

In [32]:
box1 = np.array([214,0,157.16,110.99])
box2 = np.array([216,1,158.98,104.63])
calculate_iou(box1, box2, form='coco')

0.9105388570061174

In [33]:
box1 = np.array([378,0,81.29,14.18])
box2 = np.array([367,0,73.59,12.91])
calculate_iou(box1, box2, form='coco')

0.6308332885219091

In [34]:
box1 = np.array([562,46,107.24,108.53])
box2 = np.array([563,44,102.06,111.47])
calculate_iou(box1, box2, form='coco')

0.9284155026848814

In [35]:
box1 = np.array([564,4,115.25,101.75] )
box2 = np.array([567,3,110.34,118.62])
calculate_iou(box1, box2, form='coco')

0.8276199482784963

In [36]:
box1 = np.array([648,59,102.9,100.51] )
box2 = np.array([655,55,93.08,84.62])
calculate_iou(box1, box2, form='coco')

0.7029811111368468

In [37]:
box1 = np.array([1238,2,122.62,118.39])
box2 = np.array([1242,4,108.48,113.77])
calculate_iou(box1, box2, form='coco')

0.8513467430749581

In [38]:
box1 = np.array([1333,0,124.44,114.69] )
box2 = np.array([1340,1,116.09,116.62])
calculate_iou(box1, box2, form='coco')

0.9039950603785094

In [40]:
box1 = np.array([1313,100,97.94,86.33])
box2 = np.array([1292,90,118.36,96.51])
calculate_iou(box1, box2, form='coco')

0.7348320472015555

### Image - miops-trigger-sy0HX0_mzG0-unsplash_jpg.rf.dde7f7af38757ec63b094108aed277c0.jpg

In [41]:
box1 = np.array([429,125,453.47,329.51])
box2 = np.array([429,125,456.87,326.5])
calculate_iou(box1, box2, form='coco')

0.983601318573747

In [42]:
box1 = np.array([859,12,424.62,386.1])
box2 = np.array([857,14,422.27,386.52])
calculate_iou(box1, box2, form='coco')

0.9740565228849222